In [2]:
%pip install -q sklearn

Note: you may need to restart the kernel to use updated packages.


In [4]:
from __future__ import absolute_import, division, print_function, unicode_literals

import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
from six.moves import urllib

import tensorflow.compat.v2.feature_column as fc

import tensorflow as tf

In [5]:
data_train = pd.read_csv("https://storage.googleapis.com/tf-datasets/titanic/train.csv")
data_eval = pd.read_csv("https://storage.googleapis.com/tf-datasets/titanic/eval.csv")

In [6]:
dftrain = pd.read_csv("https://storage.googleapis.com/tf-datasets/titanic/train.csv")
dfeval = pd.read_csv("https://storage.googleapis.com/tf-datasets/titanic/eval.csv")
y_train = dftrain.pop("survived")
y_eval = dfeval.pop("survived")

CATEGORICAL_COLUMNS = ["sex", "n_siblings_spouses", "parch", "class", "deck", "embark_town", "alone"]
NUMERIC_COLUMNS = ["age", "fare"]

feature_columns = []

for feature_name in CATEGORICAL_COLUMNS:
    vocabulary = dftrain[feature_name].unique()
    feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))

for feature_name in NUMERIC_COLUMNS:
    feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))

In [7]:
def make_input_fn(data_df, label_df, num_epochs=40, shuffle=True, batch_size=40):
    def input_function():
        ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))
        if shuffle:
            ds = ds.shuffle(1000)
        ds = ds.batch(batch_size).repeat(num_epochs)
        return ds
    return input_function

train_input_fn = make_input_fn(dftrain, y_train)
eval_input_fn = make_input_fn(dfeval, y_eval, num_epochs=1, shuffle=False)

linear_est = tf.estimator.LinearClassifier(feature_columns=feature_columns)


linear_est.train(train_input_fn)
result = linear_est.evaluate(eval_input_fn)

clear_output()

In [31]:
print(result["accuracy"])

0.7878788


In [9]:
try_predict = list(linear_est.predict(eval_input_fn))
clear_output()

In [ ]:
x = random.randint(0,263)
print(f"Prediction for person #{x + 1}")
print(data_eval.loc[x])
print(f"Survival percent prediction: {try_predict[x]['probabilities'][1]}")

Prediction for person #165
survived                        1
sex                          male
age                          32.0
n_siblings_spouses              0
parch                           0
fare                        7.925
class                       Third
deck                      unknown
embark_town           Southampton
alone                           y
Name: 164, dtype: object
Survival percent prediction: 0.058005400002002716
